# Setup of true test

Note: This version is against the true solution, so adaptive timestepping is used. This means that the option which is set to get the correct timepoints is saveat, not tstops!

In [13]:
using ParameterizedFunctions, OrdinaryDiffEq, DiffEqParamEstim
using BlackBoxOptim, NLopt, Plots

In [14]:
Xiang2015Bounds = Tuple{Float64, Float64}[(9, 11), (20, 30), (2, 3)] # for local optimizations
xlow_bounds = [9.0,20.0,2.0]
xhigh_bounds = [11.0,30.0,3.0]
LooserBounds = Tuple{Float64, Float64}[(0, 22), (0, 60), (0, 6)] # for global optimization
GloIniPar = [0.0, 0.5, 0.1] # for global optimizations
LocIniPar = [9.0, 20.0, 2.0] # for local optimization

3-element Array{Float64,1}:
  9.0
 20.0
  2.0

In [15]:
g1 = @ode_def_bare LorenzExample begin
  dx = σ*(y-x)
  dy = x*(ρ-z) - y
  dz = x*y - β*z
    end σ=>10.0 ρ=>28.0 β=>2.66    # Parameters used to construct the dataset

r0 = [1.0; 0.0; 0.0]                #[-11.8,-5.1,37.5] PODES Initial values of the system in space # [0.1, 0.0, 0.0]
tspan = (0.0, 50.0)                 # PODES sample of 3000 observations over the (0,30) timespan
prob = ODEProblem(g1, r0, tspan)
tspan2 = (0.0, 3.0)                 # Xiang test sample of 300 observations with a timestep of 0.01
prob_short = ODEProblem(g1, r0, tspan2)

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: [1.0, 0.0, 0.0]

In [16]:
dt = 30.0/3000                    
tf = 30.0
tinterval = 0:dt:tf
t  = collect(tinterval)

3001-element Array{Float64,1}:
  0.0 
  0.01
  0.02
  0.03
  0.04
  0.05
  0.06
  0.07
  0.08
  0.09
  0.1 
  0.11
  0.12
  ⋮   
 29.89
 29.9 
 29.91
 29.92
 29.93
 29.94
 29.95
 29.96
 29.97
 29.98
 29.99
 30.0 

In [17]:
h = 0.01
M = 300
tstart = 0.0
tstop = tstart + M * h
tinterval_short = 0:h:tstop
t_short = collect(tinterval_short)

301-element Array{Float64,1}:
 0.0 
 0.01
 0.02
 0.03
 0.04
 0.05
 0.06
 0.07
 0.08
 0.09
 0.1 
 0.11
 0.12
 ⋮   
 2.89
 2.9 
 2.91
 2.92
 2.93
 2.94
 2.95
 2.96
 2.97
 2.98
 2.99
 3.0 

In [18]:
# Generate Data
data_sol_short = solve(prob_short,Vern9(),saveat=t_short,reltol=1e-14,abstol=1e-14)
data_short = convert(Array, data_sol_short) # This operation produces column major dataset obs as columns, equations as rows
data_sol = solve(prob,Vern9(),saveat=t,reltol=1e-14,abstol=1e-14)
data = convert(Array, data_sol)

3×3002 Array{Float64,2}:
 1.0  0.917924    0.867919    0.84536     …  13.2867  12.5882    3.84682
 0.0  0.26634     0.51174     0.744654        6.7153   5.22483  -1.51867
 0.0  0.00126393  0.00465567  0.00983655     39.1674  38.9008   29.1751 

Plot the data

In [19]:
# plot(data_sol_short,vars=(1,2,3)) # the short solution
# plot(data_sol,vars=(1,2,3)) # the longer solution
#interpolation_sol = solve(prob,Vern7(),saveat=t,reltol=1e-12,abstol=1e-12)
#plot(interpolation_sol,vars=(1,2,3))

In [20]:
#xyzt = plot(data_sol_short, plotdensity=10000,lw=1.5)
#xy = plot(data_sol_short, plotdensity=10000, vars=(:x,:y))
#xz = plot(data_sol_short, plotdensity=10000, vars=(:x,:z))
#yz = plot(data_sol_short, plotdensity=10000, vars=(:y,:z))
#xyz = plot(data_sol_short, plotdensity=10000, vars=(:x,:y,:z))
#plot(plot(xyzt,xyz),plot(xy, xz, yz, layout=(1,3),w=1), layout=(2,1), size=(800,600))

In [21]:
xyzt = plot(data_sol, plotdensity=10000,lw=1.5)
xy = plot(data_sol, plotdensity=10000, vars=(:x,:y))
xz = plot(data_sol, plotdensity=10000, vars=(:x,:z))
yz = plot(data_sol, plotdensity=10000, vars=(:y,:z))
xyz = plot(data_sol, plotdensity=10000, vars=(:x,:y,:z))
plot(plot(xyzt,xyz),plot(xy, xz, yz, layout=(1,3),w=1), layout=(2,1), size=(800,600))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 t 
 
<polyline clip-path="url(#clip2702)" style="stroke:#009af9; stroke-width:1.5; stroke-opacity:1; fill:none" points="
 35.8488,176.147 35.9177,176.432 35.9866,176.606 36.0555,176.684 36.1243,176.679 36.1932,176.599 36.2621,176.45 36.331,176.236 36.3999,175.957 36.4688,175.613 
 36.5377,175.203 36.6066,174.723 36.6755,174.168 36.7444,173.532 36.8133,172.807 36.8822,171.986 36.9511,171.057 37.02,170.009 37.0889,168.83 37.1577,167.506 
 37.2266,166.021 37.2955,164.358 37.3644,162.501 37.4333,160.432 37.5022,158.133 37.5711,155.59 37.64,152.788 37.7089,149.719 37.7778,146.383 37.8467,142.79 
 37.9156,138.966 37.9845,134.957 38.0534,130.835 38.1222,126.699 38.1911,122.685 38.26,118.955 38.3289,115.7 38.3978,113.12 38.4667,111.409 38.5356,110.728 
 38.6045,111.182 38.6734,112.804 38.7423,115.547 38.8112,119.288 38.8801,123.851 38.949,129.022 39.0179,134.584 39.0868,140.328 39.1556,146.077 39.2245,151.687 
 39.2934,157.05 39.3623,162.092 39.4312,166.769 39.5001,171.06 39.569,174.962 39.6379,178.485 39.7068,181.646 39.7757,184.47 39.8446,186.985 39.9135,189.218 
 39.9824,191.198 40.0513,192.953 40.1202,194.509 40.189,195.89 40.2579,197.118 40.3268,198.215 40.3957,199.199 40.4646,200.086 40.5335,200.892 40.6024,201.628 
 40.6713,202.307 40.7402,202.938 40.8091,203.53 40.878,204.09 40.9469,204.624 41.0158,205.137 41.0847,205.633 41.1536,206.115 41.2224,206.585 41.2913,207.044 
 41.3602,207.494 41.4291,207.935 41.498,208.365 41.5669,208.785 41.6358,209.192 41.7047,209.586 41.7736,209.964 41.8425,210.323 41.9114,210.661 41.9803,210.976 
 42.0492,211.265 42.1181,211.525 42.187,211.753 42.2558,211.948 42.3247,212.107 42.3936,212.228 42.4625,212.31 42.5314,212.352 42.6003,212.354 42.6692,212.315 
 42.7381,212.236 42.807,212.118 42.8759,211.962 42.9448,211.772 43.0137,211.549 43.0826,211.296 43.1515,211.017 43.2203,210.715 43.2892,210.394 43.3581,210.059 
 43.427,209.712 43.4959,209.36 43.5648,209.005 43.6337,208.651 43.7026,208.303 43.7715,207.964 43.8404,207.637 43.9093,207.326 43.9782,207.033 44.0471,206.762 
 44.116,206.514 44.1849,206.291 44.2537,206.095 44.3226,205.928 44.3915,205.79 44.4604,205.683 44.5293,205.607 44.5982,205.563 44.6671,205.55 44.736,205.57 
 44.8049,205.62 44.8738,205.702 44.9427,205.814 45.0116,205.955 45.0805,206.124 45.1494,206.321 45.2183,206.544 45.2871,206.79 45.356,207.059 45.4249,207.347 
 45.4938,207.654 45.5627,207.976 45.6316,208.31 45.7005,208.654 45.7694,209.005 45.8383,209.359 45.9072,209.713 45.9761,210.063 46.045,210.405 46.1139,210.736 
 46.1828,211.052 46.2517,211.349 46.3205,211.623 46.3894,211.871 46.4583,212.089 46.5272,212.275 46.5961,212.425 46.665,212.536 46.7339,212.609 46.8028,212.64 
 46.8717,212.629 46.9406,212.576 47.0095,212.481 47.0784,212.346 47.1473,212.172 47.2162,211.961 47.285,211.716 47.3539,211.441 47.4228,211.138 47.4917,210.811 
 47.5606,210.465 47.6295,210.104 47.6984,209.733 47.7673,209.355 47.8362,208.975 47.9051,208.598 47.974,208.227 48.0429,207.867 48.1118,207.52 48.1807,207.19 
 48.2496,206.881 48.3184,206.594 48.3873,206.332 48.4562,206.098 48.5251,205.892 48.594,205.717 48.6629,205.573 48.7318,205.461 48.8007,205.383 48.8696,205.338 
 48.9385,205.327 49.0074,205.348 49.0763,205.403 49.1452,205.491 49.2141,205.611 49.283,205.761 49.3518,205.941 49.4207,206.15 49.4896,206.386 49.5585,206.647 
 49.6274,206.932 49.6963,207.238 49.7652,207.563 49.8341,207.904 49.903,208.259 49.9719,208.624 50.0408,208.996 50.1097,209.372 50.1786,209.748 50.2475,210.12 
 50.3164,210.484 50.3852,210.836 50.4541,211.172 50.523,211.489 50.5919,211.781 50.6608,212.045 50.7297,212.277 50.7986,212.475 50.8675,212.634 50.9364,212.754 
 51.0053,212.83 51.0742,212.863 51.1431,212.85 51.212,212.793 51.2809,212.691 51.3498,212.545 51.4186,212.358 51.487

In [22]:
# Note: Euler uses tstops to hit the estimation timepoints exactly since it's not adaptive
# obj_short = build_loss_objective(prob_short,Euler(),L2Loss(t_short,data_short),tstops=t_short)
# res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 11e3)
# Euler could not recover the correct results since its error is too high!

In [24]:
obj_short = build_loss_objective(prob_short,Tsit5(),L2Loss(t_short,data_short),tstops=t_short)
res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 7e3)
# Tolernace is still too high to get close enough

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 640 evals, 548 steps, improv/step: 0.369 (last = 0.3686), fitness=421.390158394
1.00 secs, 1310 evals, 1218 steps, improv/step: 0.325 (last = 0.2896), fitness=6.302371602
1.50 secs, 1958 evals, 1867 steps, improv/step: 0.327 (last = 0.3297), fitness=0.087344863
2.00 secs, 2589 evals, 2498 steps, improv/step: 0.316 (last = 0.2853), fitness=0.030284266
2.51 secs, 3246 evals, 3155 steps, improv/step: 0.313 (last = 0.2983), fitness=0.000743772
3.01 secs, 3907 evals, 3816 steps, improv/step: 0.312 (last = 0.3116), fitness=0.000064645
3.51 secs, 4565 evals, 4474 steps, improv/step: 0.304 (last = 0.2523), fitness=0.000001385
4.01 secs, 5226 evals, 5135 steps, improv/step: 0.302 (last = 0.2905), fitness=0.000000500
4.51

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (7000) reached", 7001, 1.515612872814e9, 5.430000066757202, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 360082),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 7000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 7090, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,A

In [26]:
obj_short = build_loss_objective(prob_short,Tsit5(),L2Loss(t_short,data_short),tstops=t_short,reltol=1e-9)
res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 7e3)
# With the tolerance lower, it achieves the correct solution in 3.5 seconds.

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 518 evals, 397 steps, improv/step: 0.370 (last = 0.3703), fitness=1232.631382075
1.00 secs, 1037 evals, 902 steps, improv/step: 0.288 (last = 0.2238), fitness=587.618526621
1.50 secs, 1550 evals, 1414 steps, improv/step: 0.291 (last = 0.2949), fitness=45.813872452
2.01 secs, 2063 evals, 1927 steps, improv/step: 0.292 (last = 0.2963), fitness=2.475955437
2.51 secs, 2555 evals, 2419 steps, improv/step: 0.289 (last = 0.2744), fitness=1.459373838
3.01 secs, 3102 evals, 2966 steps, improv/step: 0.292 (last = 0.3090), fitness=0.009273528
3.51 secs, 3640 evals, 3504 steps, improv/step: 0.292 (last = 0.2881), fitness=0.005422365
4.01 secs, 4165 evals, 4029 steps, improv/step: 0.295 (last = 0.3181), fitness=0.000647755
4

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (7000) reached", 7001, 1.515613048326e9, 6.884000062942505, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 245196),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 7000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 7135, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,A

In [29]:
obj_short = build_loss_objective(prob_short,Vern9(),L2Loss(t_short,data_short),tstops=t_short,reltol=1e-9,abstol=1e-9)
res1 = bboptimize(obj_short;SearchRange = LooserBounds, MaxSteps = 7e3)
# With the more accurate solver Vern9 in the solution of the ODE, the convergence is less efficient!

# Fastest BlackBoxOptim: 3.5 seconds

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 571 evals, 466 steps, improv/step: 0.361 (last = 0.3605), fitness=688.274201385
1.00 secs, 1154 evals, 1049 steps, improv/step: 0.323 (last = 0.2933), fitness=38.168000915
1.50 secs, 1736 evals, 1631 steps, improv/step: 0.316 (last = 0.3041), fitness=2.305451791
2.00 secs, 2323 evals, 2218 steps, improv/step: 0.320 (last = 0.3288), fitness=0.333588820
2.50 secs, 2907 evals, 2802 steps, improv/step: 0.313 (last = 0.2860), fitness=0.007907005
3.01 secs, 3495 evals, 3390 steps, improv/step: 0.311 (last = 0.3010), fitness=0.000821684
3.51 secs, 4087 evals, 3982 steps, improv/step: 0.308 (last = 0.2905), fitness=0.000089670
4.01 secs, 4671 evals, 4566 steps, improv/step: 0.310 (last = 0.3236), fitness=0.000003607
4.5

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (7000) reached", 7001, 1.515613189884e9, 6.1540000438690186, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 255002),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 7000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 7104, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,

# Using NLopt

First, the global optimization algorithms

In [30]:
obj_short = build_loss_objective(prob_short,Vern9(),L2Loss(t_short,data_short),tstops=t_short,reltol=1e-14,abstol=1e-14)

(::DiffEqObjective) (generic function with 2 methods)

In [32]:
opt = Opt(:GN_ORIG_DIRECT_L, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Accurate 3.2 seconds

  3.173465 seconds (18.15 M allocations: 812.435 MiB, 6.44% gc time)


(4.630419474729376e-19, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [33]:
opt = Opt(:GN_CRS2_LM, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Accurate 3.0 seconds

  3.010769 seconds (16.69 M allocations: 746.735 MiB, 6.05% gc time)


(1.4250715551452745e-19, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [34]:
opt = Opt(:GN_ISRES, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Accurate to single precision 8.2 seconds

  8.212183 seconds (47.51 M allocations: 2.076 GiB, 6.21% gc time)


(0.01795283396918925, [9.99834, 28.0047, 2.65955], :MAXEVAL_REACHED)

In [35]:
opt = Opt(:GN_ESCH, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Approximatively accurate, good starting values for local optimization

  8.231527 seconds (47.51 M allocations: 2.076 GiB, 6.20% gc time)


(83.89691323804982, [9.75945, 28.227, 2.59969], :MAXEVAL_REACHED)

Next, the local optimization algorithms that could be used after the global algorithms as a check on the solution and its precision. All the local optimizers are started from LocIniPar and with the narrow bounds of the Xiang2015Paper. 

In [37]:
opt = Opt(:LN_BOBYQA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # 0.1 seconds

  0.112560 seconds (527.64 k allocations: 23.617 MiB, 13.38% gc time)


(3.8076007012492644e-23, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [38]:
opt = Opt(:LN_NELDERMEAD, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.29 sec

  0.288990 seconds (1.56 M allocations: 69.741 MiB, 5.76% gc time)


(9.409888589748689e-21, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [39]:
opt = Opt(:LD_SLSQP, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.21 sec

  0.212325 seconds (851.00 k allocations: 38.170 MiB, 7.66% gc time)


(1.1132273506594414e-15, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [40]:
opt = Opt(:LN_COBYLA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 1.84 sec

  1.830445 seconds (10.35 M allocations: 463.307 MiB, 6.30% gc time)


(2.5482220525708953e-19, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [41]:
opt = Opt(:LN_NEWUOA_BOUND, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.40 sec ROUNDOFF LIMITED

  0.405937 seconds (1.58 M allocations: 70.804 MiB, 4.19% gc time)


(7.532358644244913e-10, [10.0, 28.0, 2.66], :SUCCESS)

In [42]:
opt = Opt(:LN_PRAXIS, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.18 sec

  0.181651 seconds (869.45 k allocations: 38.911 MiB, 7.27% gc time)


(1.3975550138943548e-21, [10.0, 28.0, 2.66], :SUCCESS)

In [43]:
opt = Opt(:LN_SBPLX, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.65 sec

  0.650668 seconds (3.61 M allocations: 161.594 MiB, 5.82% gc time)


(3.604136287129055e-21, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [44]:
opt = Opt(:LD_MMA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.7 sec

  0.696521 seconds (3.92 M allocations: 175.584 MiB, 6.25% gc time)


(2.4906192350030634e-16, [10.0, 28.0, 2.66], :XTOL_REACHED)

In [45]:
opt = Opt(:LD_LBFGS, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.11 sec

  0.115152 seconds (598.57 k allocations: 26.784 MiB, 4.92% gc time)


(1.1132796234229577e-15, [10.0, 28.0, 2.66], :SUCCESS)

In [47]:
opt = Opt(:LD_TNEWTON_PRECOND_RESTART, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj_short.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Accurate 0.15 sec

  0.148135 seconds (764.83 k allocations: 34.224 MiB, 9.54% gc time)


(1.1134972923032622e-15, [10.0, 28.0, 2.66], :SUCCESS)


## Now let's solve the longer version

Notice from the plotting above that this ODE problem is chaotic and tends to diverge over time. In the longer version of parameter estimation, the dataset is increased to 3000 observations per variable with the same integration time step of 0.01.

In [49]:
#obj = build_loss_objective(prob,Euler(),L2Loss(t,data),tstops=t)
#res1 = bboptimize(obj;SearchRange = LooserBounds, MaxSteps = 4e3)
# Once again, Euler fails to converge. Error cannot be reduced.

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps


0.50 secs, 92 evals, 52 steps, improv/step: 0.404 (last = 0.4038), fitness=883463.043909482
1.01 secs, 188 evals, 115 steps, improv/step: 0.443 (last = 0.4762), fitness=755472.526475439


1.51 secs, 281 evals, 193 steps, improv/step: 0.492 (last = 0.5641), fitness=735005.493557285
2.01 secs, 373 evals, 279 steps, improv/step: 0.459 (last = 0.3837), fitness=719003.243335392
2.52 secs, 466 evals, 366 steps, improv/step: 0.443 (last = 0.3908), fitness=719003.243335392
3.02 secs, 560 evals, 459 steps, improv/step: 0.420 (last = 0.3333), fitness=719003.243335392
3.52 secs, 655 evals, 554 steps, improv/step: 0.408 (last = 0.3474), fitness=719003.243335392
4.02 secs, 749 evals, 649 steps, improv/step: 0.414 (last = 0.4526), fitness=719003.243335392
4.53 secs, 843 evals, 743 steps, improv/step: 0.401 (last = 0.3085), fitness=719003.243335392
5.03 secs, 937 evals, 837 steps, improv/step: 0.403 (last = 0.4149), fitness=719003.243335392
5.54 secs, 1031 evals, 931 steps, improv/step: 0.396 (last = 0.3404), fitness=719003.243335392
6.04 secs, 1124 evals, 1024 steps, improv/step: 0.396 (last = 0.3978), fitness=719003.243335392
6.54 secs, 1218 evals, 1118 steps, improv/step: 0.395 (la

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (4000) reached", 4001, 1.515576369776e9, 22.76900005340576, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 897330),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 4000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 4101, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,A

In [48]:
obj = build_loss_objective(prob,Vern9(),L2Loss(t,data),tstops=t,reltol=1e-14,abstol=1e-14)
res1 = bboptimize(obj;SearchRange = LooserBounds, MaxSteps = 40e3)
# BB with Vern9 converges very slowly. The final values are within the NarrowBounds.

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.55 secs, 24 evals, 12 steps, improv/step: 0.667 (last = 0.6667), fitness=705752.351552874
1.05 secs, 48 evals, 25 steps, improv/step: 0.600 (last = 0.5385), fitness=563038.384260488
1.58 secs, 73 evals, 40 steps, improv/step: 0.550 (last = 0.4667), fitness=563038.384260488
2.11 secs, 99 evals, 56 steps, improv/step: 0.536 (last = 0.5000), fitness=563038.384260488
2.64 secs, 123 evals, 70 steps, improv/step: 0.514 (last = 0.4286), fitness=563038.384260488
3.16 secs, 146 evals, 87 steps, improv/step: 0.506 (last = 0.4706), fitness=554726.124829995
3.66 secs, 169 evals, 106 steps, improv/step: 0.509 (last = 0.5263), fitness=554726.124829995
4.20 secs, 194 evals, 124 steps, improv/step: 0.492 (last = 0.3889), fitness=554726.

43.11 secs, 1921 evals, 1820 steps, improv/step: 0.218 (last = 0.0833), fitness=504955.318555998
43.63 secs, 1944 evals, 1843 steps, improv/step: 0.218 (last = 0.2174), fitness=504955.318555998
44.13 secs, 1967 evals, 1866 steps, improv/step: 0.217 (last = 0.1304), fitness=504955.318555998
44.65 secs, 1990 evals, 1889 steps, improv/step: 0.216 (last = 0.1739), fitness=504039.083006339
45.17 secs, 2013 evals, 1912 steps, improv/step: 0.215 (last = 0.1304), fitness=504039.083006339
45.69 secs, 2036 evals, 1935 steps, improv/step: 0.214 (last = 0.1304), fitness=504039.083006339
46.21 secs, 2058 evals, 1957 steps, improv/step: 0.215 (last = 0.2727), fitness=504039.083006339
46.73 secs, 2080 evals, 1979 steps, improv/step: 0.213 (last = 0.0909), fitness=504039.083006339
47.24 secs, 2103 evals, 2002 steps, improv/step: 0.212 (last = 0.1304), fitness=504039.083006339
47.76 secs, 2125 evals, 2024 steps, improv/step: 0.212 (last = 0.1818), fitness=504039.083006339
48.27 secs, 2144 evals, 2043 s

86.61 secs, 3777 evals, 3677 steps, improv/step: 0.166 (last = 0.1053), fitness=470533.930424762
87.12 secs, 3794 evals, 3694 steps, improv/step: 0.166 (last = 0.0000), fitness=470533.930424762
87.63 secs, 3813 evals, 3713 steps, improv/step: 0.165 (last = 0.1053), fitness=470533.930424762
88.14 secs, 3833 evals, 3733 steps, improv/step: 0.165 (last = 0.1500), fitness=470533.930424762
88.66 secs, 3849 evals, 3749 steps, improv/step: 0.165 (last = 0.0625), fitness=470533.930424762
89.19 secs, 3864 evals, 3764 steps, improv/step: 0.164 (last = 0.0000), fitness=470533.930424762
89.70 secs, 3885 evals, 3785 steps, improv/step: 0.164 (last = 0.0952), fitness=470533.930424762
90.22 secs, 3907 evals, 3807 steps, improv/step: 0.164 (last = 0.1364), fitness=470533.930424762
90.74 secs, 3928 evals, 3828 steps, improv/step: 0.164 (last = 0.1429), fitness=470533.930424762
91.26 secs, 3950 evals, 3850 steps, improv/step: 0.163 (last = 0.0909), fitness=470533.930424762
91.77 secs, 3972 evals, 3872 s

129.74 secs, 5572 evals, 5472 steps, improv/step: 0.147 (last = 0.0952), fitness=443865.787733175
130.25 secs, 5588 evals, 5488 steps, improv/step: 0.147 (last = 0.0000), fitness=443865.787733175
130.75 secs, 5610 evals, 5510 steps, improv/step: 0.147 (last = 0.2273), fitness=443865.787733175
131.26 secs, 5631 evals, 5531 steps, improv/step: 0.148 (last = 0.2857), fitness=443865.787733175
131.78 secs, 5650 evals, 5550 steps, improv/step: 0.148 (last = 0.1579), fitness=443865.787733175
132.30 secs, 5669 evals, 5569 steps, improv/step: 0.148 (last = 0.1579), fitness=434802.572528226
132.80 secs, 5690 evals, 5590 steps, improv/step: 0.148 (last = 0.1905), fitness=434802.572528226
133.31 secs, 5712 evals, 5612 steps, improv/step: 0.147 (last = 0.0000), fitness=434802.572528226
133.84 secs, 5734 evals, 5634 steps, improv/step: 0.147 (last = 0.0455), fitness=434802.572528226
134.35 secs, 5755 evals, 5655 steps, improv/step: 0.147 (last = 0.1429), fitness=434802.572528226
134.87 secs, 5778 ev

172.70 secs, 7364 evals, 7265 steps, improv/step: 0.127 (last = 0.0909), fitness=425215.533283514
173.21 secs, 7385 evals, 7286 steps, improv/step: 0.127 (last = 0.0476), fitness=425215.533283514
173.73 secs, 7407 evals, 7308 steps, improv/step: 0.126 (last = 0.0000), fitness=425215.533283514
174.24 secs, 7430 evals, 7331 steps, improv/step: 0.126 (last = 0.0000), fitness=425215.533283514
174.75 secs, 7452 evals, 7353 steps, improv/step: 0.126 (last = 0.0909), fitness=425215.533283514
175.26 secs, 7474 evals, 7375 steps, improv/step: 0.125 (last = 0.0455), fitness=425215.533283514
175.77 secs, 7496 evals, 7397 steps, improv/step: 0.125 (last = 0.0455), fitness=425215.533283514
176.28 secs, 7518 evals, 7419 steps, improv/step: 0.125 (last = 0.0000), fitness=425215.533283514
176.80 secs, 7539 evals, 7440 steps, improv/step: 0.124 (last = 0.0000), fitness=425215.533283514
177.31 secs, 7560 evals, 7461 steps, improv/step: 0.124 (last = 0.0000), fitness=425215.533283514
177.82 secs, 7579 ev

...

300.32 secs, 12573 evals, 12475 steps, improv/step: 0.088 (last = 0.0000), fitness=346932.713993497
300.83 secs, 12593 evals, 12495 steps, improv/step: 0.088 (last = 0.0000), fitness=346932.713993497
301.33 secs, 12612 evals, 12514 steps, improv/step: 0.088 (last = 0.0000), fitness=346932.713993497
301.84 secs, 12633 evals, 12535 steps, improv/step: 0.088 (last = 0.0000), fitness=346932.713993497
302.35 secs, 12655 evals, 12557 steps, improv/step: 0.088 (last = 0.0455), fitness=346932.713993497
302.87 secs, 12677 evals, 12579 steps, improv/step: 0.088 (last = 0.0000), fitness=346932.713993497
303.39 secs, 12699 evals, 12601 steps, improv/step: 0.088 (last = 0.0455), fitness=346932.713993497
303.91 secs, 12721 evals, 12623 steps, improv/step: 0.088 (last = 0.0000), fitness=346932.713993497
304.42 secs, 12743 evals, 12645 steps, improv/step: 0.087 (last = 0.0000), fitness=346932.713993497
304.93 secs, 12765 evals, 12667 steps, improv/step: 0.087 (last = 0.0000), fitness=346932.713993497


342.40 secs, 14315 evals, 14218 steps, improv/step: 0.081 (last = 0.0455), fitness=346932.713993497
342.91 secs, 14336 evals, 14239 steps, improv/step: 0.081 (last = 0.0476), fitness=346932.713993497
343.43 secs, 14357 evals, 14260 steps, improv/step: 0.081 (last = 0.0000), fitness=346932.713993497
343.95 secs, 14378 evals, 14281 steps, improv/step: 0.080 (last = 0.0000), fitness=346932.713993497
344.46 secs, 14400 evals, 14303 steps, improv/step: 0.080 (last = 0.0000), fitness=346932.713993497
344.97 secs, 14422 evals, 14325 steps, improv/step: 0.080 (last = 0.0909), fitness=346932.713993497
345.49 secs, 14443 evals, 14346 steps, improv/step: 0.080 (last = 0.0952), fitness=346932.713993497
346.00 secs, 14465 evals, 14368 steps, improv/step: 0.080 (last = 0.0455), fitness=346932.713993497
346.51 secs, 14486 evals, 14390 steps, improv/step: 0.080 (last = 0.0909), fitness=346932.713993497
347.02 secs, 14508 evals, 14413 steps, improv/step: 0.080 (last = 0.0435), fitness=346932.713993497


384.48 secs, 16039 evals, 16248 steps, improv/step: 0.078 (last = 0.1739), fitness=346205.226299008
384.99 secs, 16061 evals, 16272 steps, improv/step: 0.078 (last = 0.0833), fitness=346205.226299008
385.49 secs, 16083 evals, 16299 steps, improv/step: 0.078 (last = 0.1111), fitness=346205.226299008
386.00 secs, 16105 evals, 16324 steps, improv/step: 0.078 (last = 0.1600), fitness=346205.226299008
386.51 secs, 16127 evals, 16348 steps, improv/step: 0.078 (last = 0.0417), fitness=346205.226299008
387.02 secs, 16147 evals, 16372 steps, improv/step: 0.078 (last = 0.1250), fitness=346205.226299008
387.52 secs, 16169 evals, 16399 steps, improv/step: 0.078 (last = 0.1852), fitness=346205.226299008
388.05 secs, 16191 evals, 16425 steps, improv/step: 0.078 (last = 0.1154), fitness=346205.226299008
388.55 secs, 16213 evals, 16449 steps, improv/step: 0.078 (last = 0.2083), fitness=346205.226299008
389.05 secs, 16235 evals, 16480 steps, improv/step: 0.078 (last = 0.0645), fitness=346205.226299008


426.36 secs, 17780 evals, 18625 steps, improv/step: 0.080 (last = 0.0455), fitness=346190.977728164
426.87 secs, 17802 evals, 18662 steps, improv/step: 0.080 (last = 0.0270), fitness=346190.977728164
427.37 secs, 17824 evals, 18697 steps, improv/step: 0.080 (last = 0.0857), fitness=346190.977728164
427.88 secs, 17846 evals, 18737 steps, improv/step: 0.080 (last = 0.0750), fitness=346190.977728164
428.39 secs, 17868 evals, 18775 steps, improv/step: 0.079 (last = 0.0000), fitness=346190.977728164
428.90 secs, 17890 evals, 18816 steps, improv/step: 0.079 (last = 0.0488), fitness=346190.977728164
429.40 secs, 17912 evals, 18862 steps, improv/step: 0.079 (last = 0.0217), fitness=346190.977728164
429.91 secs, 17933 evals, 18914 steps, improv/step: 0.079 (last = 0.0385), fitness=346190.977728164
430.42 secs, 17955 evals, 18970 steps, improv/step: 0.079 (last = 0.0357), fitness=346190.977728164
430.93 secs, 17977 evals, 19025 steps, improv/step: 0.079 (last = 0.0364), fitness=346190.977728164


BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Too many steps (101) without any function evaluations (probably search has converged)", 19964, 1.515614324703e9, 436.80099987983704, BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 331318),Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(0.0, 22.0), (0.0, 60.0), (0.0, 6.0)]),Pair{Symbol,Any}(:MaxSteps, 40000)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,Any}(:MaxTime, 0.0),Pair{Symbol,Any}(:SearchRange, (-1.0, 1.0)),Pair{Symbol,Any}(:Method, :adaptive_de_rand_1_bin_radiuslimited),Pair{Symbol,Any}(:MaxNumStepsWithoutFuncEvals, 100),Pair{Symbol,Any}(:RngSeed, 1234),Pair{Symbol,Any}(:MaxFuncEvals, 0),Pair{Symbol,Any}(:SaveTrace, false)…)], 18228, BlackBoxOptim.ScalarFitnessSchem

In [49]:
opt = Opt(:GN_ORIG_DIRECT_L, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Fail to converge 

 43.056589 seconds (104.19 M allocations: 3.735 GiB, 2.03% gc time)


(470335.3686765812, [7.04703, 23.6653, 1.80658], :XTOL_REACHED)

In [50]:
opt = Opt(:GN_CRS2_LM, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Fail to converge approximately accurate vales for local opt.29.48 seconds

220.782795 seconds (527.54 M allocations: 18.909 GiB, 2.01% gc time)


(367498.3441511505, [7.5424, 25.2969, 1.93905], :MAXEVAL_REACHED)

In [51]:
opt = Opt(:GN_ISRES, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 50000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Approximately accurate within local bounds

1137.297452 seconds (2.64 G allocations: 94.545 GiB, 2.20% gc time)


(375304.139195853, [13.265, 25.5677, 2.53275], :MAXEVAL_REACHED)

In [53]:
opt = Opt(:GN_ESCH, 3)
lower_bounds!(opt,[0.0,0.0,0.0])
upper_bounds!(opt,[22.0,60.0,6.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 20000)
@time (minf,minx,ret) = NLopt.optimize(opt,GloIniPar) # Approximately accurate

440.101099 seconds (1.06 G allocations: 37.818 GiB, 2.02% gc time)


(475292.33777085424, [12.8084, 26.1139, 2.7875], :MAXEVAL_REACHED)

This parameter estimation on the longer sample proves to be extremely challenging for the global optimizers. BlackBoxOptim is best in optimizing the objective function. All of the global algorithms produces final parameter estimates that could be used as starting values for further refinement with the local optimization algorithms.

In [54]:
opt = Opt(:LN_BOBYQA, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Claims SUCCESS but does not iterate to the true values.

  2.334053 seconds (5.70 M allocations: 209.118 MiB, 1.94% gc time)


(588301.876527702, [9.86631, 20.5815, 2.0], :SUCCESS)

In [55]:
opt = Opt(:LN_NELDERMEAD, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-9)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Inaccurate final values

  4.255017 seconds (9.65 M allocations: 354.338 MiB, 2.21% gc time)


(439664.3609672592, [9.9937, 23.2594, 2.15897], :XTOL_REACHED)

In [56]:
opt = Opt(:LD_SLSQP, 3)
lower_bounds!(opt,[9.0,20.0,2.0])
upper_bounds!(opt,[11.0,30.0,3.0])
min_objective!(opt, obj.cost_function2)
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
@time (minf,minx,ret) = NLopt.optimize(opt,LocIniPar) # Inaccurate final values

  1.306649 seconds (3.01 M allocations: 110.365 MiB, 2.01% gc time)


(591291.6479752676, [9.1791, 20.6017, 2.0338], :XTOL_REACHED)

No local optimizer can improve the global solution to the true values.

# Conclusion:

 1) To benchmark parameter estimators, make sure the synthetic data is correct.
    The lorenz_test.jl results are all skewed because the actual Euler solution
    producing the synthetic data was skewed. Euler being a fixed timestep, it was able
    to rediscover the parameters that built the data, given that it's hitting all the
    same points. In this experiemnt this is no longer the case. There is adaptive timestepping with Vern7,
    we're getting rid of the ability to "overfit", i.e. simply match the data
    we generated. Instead, we have to hit the same solution, albeit at different
    stepping points, making it less prone to generating bias.
    
 2) As expected the Lorenz system is extremely sensitive to initial space values. 
    Starting the integration from r0 = [0.1,0.0,0.0] produces convergence with the short
    sample of 300 observations. This can be achieved by all the global optimizers as well as most of the local optimizers. 
    Instead starting from r0= [-11.8,-5.1,37.5], as in PODES, with the shorter sample shrinks
    the number of successful algorithms to 3: BBO, :GN_CRS2_LM and :LD_SLSQP. For the longer sample, all the algorithms fail.
    THIS IS A FAILURE OF THE NONLINEAR LEAST SQUARES ESTIMATOR L2LOSS.
    
 3) When trying to hit the real data, having a low enough tolerance on the
    numerical solution is key. If the numerical solution is too rough, then
    we can never actually hone in on the true parameters since even with the
    true parameters we will erroneously induce numerical error. Maybe this
    could be adaptive?
    
 4) Excessively low tolerance in the numerical solution is inefficient and delays the convergence of the estimation.
 
 5) The estimation method and the glocal versus local optimization make a huge difference in the timings. Here, BBO
    always find the correct solution for a global optimization setup. For local optimization, most methods in NLopt,
    like :LN_BOBYQA, solve the problem in <0.05 seconds. This is an algorithm that can scale a local optimization
    but we are aiming to scale a global optimization.
    
 6) Fitting shorter timespans is easier... maybe this can lead to determining a minimal sample size for the 
    optimizers and the estimator to succeed.